In [1]:
import numpy as np

In [4]:
# 4차원 데이터

x = np.random.rand(10, 1, 28, 28)  # 높이28, 너비28, 채널 1개인, 데이터 10개
x.shape

(10, 1, 28, 28)

In [5]:
# 첫번째 데이터와 2번째 데이터의 shape

print(x[0].shape)
print(x[1].shape)

(1, 28, 28)
(1, 28, 28)


In [8]:
# 첫번째 데이터의 첫번쨰 채널

x[0][0]

array([[5.06845279e-01, 3.42588380e-01, 6.78287264e-01, 7.80573653e-01,
        1.51772599e-02, 3.66912034e-01, 6.87455364e-01, 3.97101458e-01,
        7.22326524e-01, 6.63328583e-01, 6.58788316e-01, 2.55098231e-01,
        3.85257108e-01, 8.84526770e-01, 7.99374603e-01, 3.36824382e-01,
        3.01354142e-01, 5.93306268e-01, 3.16025120e-01, 8.35304338e-01,
        8.07771249e-01, 4.35812885e-01, 8.33202321e-01, 5.78919654e-01,
        4.19381759e-01, 6.77232176e-01, 2.06515706e-01, 4.53645350e-01],
       [3.02384962e-02, 8.67789567e-01, 5.80835107e-01, 7.17469878e-01,
        1.81600805e-01, 6.30149592e-01, 4.79699807e-01, 8.03070232e-01,
        1.24940512e-01, 2.56279015e-01, 3.36682752e-01, 5.62277542e-01,
        1.75130260e-01, 1.84269995e-01, 6.55449165e-01, 5.50843464e-01,
        5.52274431e-01, 5.21741124e-01, 9.75309452e-01, 1.50343021e-02,
        8.75267353e-01, 2.73453904e-01, 7.86929521e-01, 1.89267194e-01,
        5.69825714e-01, 3.53605449e-01, 9.50782088e-01, 9.75043

In [9]:
from data.util import im2col

x1 = np.random.rand(1,3,7,7)  # (데이터 수, 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0) 
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7) # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [10]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W  # 필터(가중치)
        self.b = b  # 편향
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T  # 필터 전개
        out = np.dot(col, col_W) + self.b

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

In [11]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        # 전개
        col = im2col(x, out_h, out_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        # 최댓값
        out = np.max(col, axis=1)

        # 성형
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

In [12]:
from data.functions import *
from data.layers import *
from collections import OrderedDict

In [13]:
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28),
                conv_param={'fiter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                hidden_size = 100, output_size=10, weight_init_std=0.01):
        
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        # CNN layer
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'],
                                           self.params['b1'],
                                           conv_param['stride'],
                                           conv_param['pad'])
        
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()

In [14]:
def predict(self, x):
    for layer in self.layers.values():
        x = layer.forward(x)
    return x

def loss(self, x, t):
    y = self.predict(x)
    return self.last_layer.forward(y, t)

In [16]:
def gradient(self, x, t):

    # forward
    self.loss(x, t)

    # backward
    dout = 1
    dout = self.last_layer.backward(dout)

    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
        dout = layer.backward(dout)
    
    # result save
    grads = {}
    grads['W1'] = self.layers['Conv1'].dW
    grads['b1'] = self.layers['Conv1'].db
    grads['W2'] = self.layers['Conv1'].dW
    grads['b2'] = self.layers['Conv1'].db
    grads['W3'] = self.layers['Conv1'].dW
    grads['b3'] = self.layers['Conv1'].db
